In [ ]:
#Importar librerias relevantes para el ETL

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
df_calendar = pd.read_csv('data/calendar.csv', sep = ',', index_col= 'listing_id')
df_listings = pd.read_csv('data/listings.csv', sep = ',', index_col= 'id', low_memory=False)
df_reviews = pd.read_csv('data/reviews.csv', sep = ',', index_col= 'listing_id')

## Dataframe Listings

In [36]:
df_listings.describe()

,host_id,host_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,guests_included,availability_30,availability_60,availability_90,availability_365,number_of_reviews,reviews_per_month
count,2.372900e+04,23726.000000,23729.000000,23729.000000,23729.000000,23667.000000,23686.000000,23562.000000,23729.000000,23729.000000,23729.000000,23729.000000,23729.000000,23729.00000,17222.000000
mean,1.052637e+08,9.765869,-34.592575,-58.415900,2.761768,1.259243,1.135988,1.863085,1.377934,19.494711,41.351384,63.909225,209.383750,16.31333,1.053696
std,9.990251e+07,70.044391,0.018282,0.029925,1.480604,0.631629,0.859007,1.486757,0.947027,12.985137,24.820986,35.916486,137.991373,32.32226,1.158269
min,2.616000e+03,0.000000,-34.688950,-58.530200,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.010000
25%,1.546679e+07,1.000000,-34.603430,-58.435380,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,19.000000,36.000000,89.000000,0.00000,0.230000
50%,6.886052e+07,1.000000,-34.591890,-58.415160,2.000000,1.000000,1.000000,1.000000,1.000000,28.000000,58.000000,87.000000,180.000000,4.00000,0.640000
75%,1.841516e+08,4.000000,-34.582150,-58.391750,4.000000,1.500000,1.000000,2.000000,1.000000,30.000000,60.000000,90.000000,363.000000,18.00000,1.470000
max,3.445900e+08,1511.000000,-34.534100,-58.355410,16.000000,30.000000,30.000000,50.000000,16.000000,30.000000,60.000000,90.000000,365.000000,500.00000,10.730000


In [37]:

df_listings.isna().sum().sort_values(ascending = False,)

cleaning_fee                        6785
reviews_per_month                   6507
host_acceptance_rate                5199
description                          801
city                                 587
beds                                 167
bathrooms                             62
bedrooms                              43
name                                  10
host_is_superhost                      3
host_identity_verified                 3
host_name                              3
host_since                             3
host_has_profile_pic                   3
host_listings_count                    3
is_location_exact                      0
has_availability                       0
require_guest_phone_verification       0
require_guest_profile_picture          0
cancellation_policy                    0
instant_bookable                       0
number_of_reviews                      0
availability_365                       0
availability_90                        0
availability_60 

In [33]:
columnas_a_eliminar = [
    "listing_url", "scrape_id", "last_scraped", "space", "summary", "experiences_offered",
    "neighborhood_overview", "notes", "transit", "access", "interaction", "house_rules",
    "thumbnail_url", "medium_url", "picture_url", "xl_picture_url", "host_url", "host_about",
    "host_thumbnail_url", "host_picture_url", "host_total_listings_count", "host_verifications",
    "street", "neighbourhood", "neighbourhood_group_cleansed", "state", "zipcode", "market",
    "smart_location", "country_code", "country", "square_feet", "weekly_price", "monthly_price",
    "minimum_minimum_nights", "maximum_minimum_nights", "minimum_maximum_nights",
    "maximum_maximum_nights", "minimum_nights_avg_ntm", "maximum_nights_avg_ntm",
    "calendar_updated", "calendar_last_scraped", "number_of_reviews_ltm",
    "first_review", "last_review", "requires_license", "license", "jurisdiction_names",
    "is_business_travel_ready", "calculated_host_listings_count",
    "calculated_host_listings_count_entire_homes", "calculated_host_listings_count_private_rooms",
    "calculated_host_listings_count_shared_rooms", "host_location", "host_neighbourhood",
    "maximum_nights", "minimum_nights", "security_deposit", "review_scores_value", "review_scores_communication",
    "review_scores_cleanliness", "review_scores_location", "review_scores_checkin", "review_scores_accuracy",
    "review_scores_rating", "host_response_time", "host_response_rate"             
]

# Eliminar columnas en la lista "columnas_a_eliminar"
df_listings = df_listings.drop(columns=columnas_a_eliminar)

In [ ]:
df_reviews.info()

In [ ]:

df_listings.isna().sum().sort_values(ascending = False,)